In [1]:
## PART 3
import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'mehdi',
    'password': 'Sz2295780211',
    'host': 'localhost',
    'database': 'data',
    'auth_plugin' : 'mysql_native_password'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

In [2]:
def markdown_table(res, titles):
    if len(titles) != len(res[0]):
        raise Exception("arg arrays don't match")

    table = ' | '.join(titles)
    table += '\n'
    table += ' | '.join([":---:"] * len(titles))
    table += '\n'

    for row in res:
        table += ' | '.join([str(elem) for elem in row])
        table += '\n'

    return table

In [51]:
##QUERY2
cursor.execute("""SELECT vm.description , COUNT(vm.id)
FROM(
    SELECT p.id 
    FROM Collisions c, Factors f, RoadCondition r, Parties p
    WHERE c.case_id = f.case_id AND( 
        r.id = f.road_condition_1 OR 
        r.id = f.road_condition_2) AND
        r.description = 'holes' AND
        c.case_id = p.case_id ) AS parties_holes , Vehicles v , VehiculeMake vm
WHERE parties_holes.id = v.party_id AND
      vm.id = v.vehicle_make
GROUP BY vm.description
ORDER BY COUNT(vm.id) DESC
LIMIT 5
      """)

In [52]:
res = cursor.fetchall()

In [53]:
res

[('FORD', 2822),
 ('TOYOTA', 2145),
 ('HONDA', 1824),
 ('CHEVROLET', 1816),
 ('NISSAN', 775)]

In [76]:
###QUERY 3
cursor.execute("""SELECT vm.description , COUNT(vm.description)
FROM(
    SELECT p.id 
    FROM Collisions c , CollisionSeverity cs, Parties p
    WHERE c.collision_severity = cs.id AND ( 
          cs.description = 'Fatal' OR
          cs.description = 'Severe') AND 
          p.case_id = c.case_id) AS hard_collisions, Vehicles v, VehiculeMake vm
WHERE v.party_id = hard_collisions.id AND   
      vm.id = v.vehicle_make
GROUP BY (vm.description)
ORDER BY COUNT(vm.description) DESC 
LIMIT 10
""")


In [77]:
res = cursor.fetchall()

In [78]:
 for x in res :
     print(x)

('FORD', 6581)
('CHEVROLET', 4901)
('TOYOTA', 4333)
('HONDA', 3889)
('DODGE', 1990)
('NISSAN', 1626)
('GMC', 1083)
('NOT STATED', 927)
('MISCELLANEOUS', 822)
('FREIGHTLINER', 810)


In [25]:
###QUERY 4
cursor.execute("""SET @v1:=(SELECT COUNT(*)
                            FROM Collisions c, Victims v ,VictimDegreeOfInjury vd
                            WHERE c.case_id = v.case_id                     AND 
                                    v.victim_degree_of_injury = vd.id         AND
                                    vd.description = 'No Injury' )          
                            """)
cursor.execute("""
    CREATE VIEW Position_Safety AS (
        SELECT  v.victim_seating_position, COUNT(*) as pourcentage
        FROM Collisions c, Victims v ,VictimDegreeOfInjury vd
        WHERE c.case_id = v.case_id                     AND 
            v.victim_degree_of_injury = vd.id         AND
            vd.description = 'No Injury'              AND
            v.victim_seating_position IS NOT NULL
            
        GROUP BY v.victim_seating_position
        ORDER BY COUNT(*) DESC ) 
""")


In [43]:
cursor.execute("""
SELECT victim_seating_position, pourcentage/@v1 
FROM Position_Safety  
WHERE pourcentage/@v1 = (SELECT MAX(pourcentage/@v1)FROM Position_Safety) OR 
      pourcentage/@v1 = (SELECT MIN(pourcentage/@v1)FROM Position_Safety) 
""")

In [44]:
res = cursor.fetchall()


In [45]:
for x in res:
    print (x)

(3, Decimal('0.4812'))
(1, Decimal('0.0059'))


In [46]:
print(markdown_table(res,['Seating','Safety %']))

Seating | Safety %
:---: | :---:
3 | 0.4812
1 | 0.0059



In [43]:
#QUERY5
cursor.execute("""
SET @cities:=(
SELECT COUNT(DISTINCT county_city_location)
FROM Locations)
                """)
cursor.execute(""" SELECT @cities""")
res=cursor.fetchall()






In [44]:
print(res)

[(540,)]


In [37]:

cursor.execute("""
SELECT COUNT(*)
FROM Collisions

                """)

In [38]:
res = cursor.fetchall()
print(res)

[(3678062,)]


In [45]:
cursor.execute("""
SELECT COUNT(*)
FROM (SELECT vehicle_make ,COUNT(Parties.case_id) as collisions ,COUNT(DISTINCT county_city_location) as city
      FROM (Vehicles   INNER JOIN Parties ON Parties.id = Vehicles.party_id 
                       INNER JOIN Locations ON Locations.case_id = Parties.id) 
GROUP BY vehicle_make
) A
WHERE A.collisions > 10 AND
      A.city > @cities/2
""")

In [46]:
res = cursor.fetchall()

In [47]:
print(res)

[(65,)]


In [27]:
cursor.execute("""

       SELECT l1.population
       FROM Locations l1
       ORDER BY l1.population ASC
       LIMIT 3

                """)
#cursor.execute(""" SELECT @population""")
res=cursor.fetchall()



In [8]:
###QUERY6
cursor.execute("""

    SELECT l.county_city_location , l.population , l.case_id
    FROM Locations l
    WHERE l.population <= @population
       
""")

InternalError: Unread result found

In [28]:
print(res)

, (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4

In [20]:
print(res)

[(9,)]
